In [1]:
%matplotlib inline
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split,ShuffleSplit
from sklearn.metrics import r2_score
from sklearn import ensemble
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [125]:
# ignore warnings from pandas
import warnings
warnings.filterwarnings('ignore')

In [167]:
df_train_raw = pd.read_csv("./data/train.csv")
df_test_raw = pd.read_csv("./data/data_for_test.csv")

In [ ]:
#df_train = df_train_raw.groupby(['key']).mean().reset_index()

In [168]:
df_train_raw['xx1_roll'] = df_train_raw['xx1'].rolling(3).median()
df_train_raw['xx2_roll'] = df_train_raw['xx2'].rolling(3).median() 
df_train_raw['xx3_roll'] = df_train_raw['xx3'].rolling(3).median() 
df_train_raw['xx4_roll'] = df_train_raw['xx4'].rolling(3).median() 
df_train_raw['xx5_roll'] = df_train_raw['xx5'].rolling(3).median() 

In [169]:
df_train_raw.head()

,patient_id,key,gender,age,x1,x2,x3,x4,x5,x6,...,xx3,xx4,xx5,y_mean_MAP,y_mean_HR,xx1_roll,xx2_roll,xx3_roll,xx4_roll,xx5_roll
0,1891,1891-3,1,75,22,1,2,0,0,1,...,99.9,49.4,62.5,59.04,80.0,NaN,NaN,NaN,NaN,NaN
1,1891,1891-3,1,75,22,1,2,0,0,1,...,98.4,48.7,61.3,59.04,80.0,NaN,NaN,NaN,NaN,NaN
2,1891,1891-3,1,75,22,1,2,0,0,1,...,95.3,48.5,60.3,59.04,80.0,80.0,95.0,98.4,48.7,61.3
3,1891,1891-3,1,75,22,1,2,0,0,1,...,97.4,48.9,61.3,59.04,80.0,80.0,95.0,97.4,48.7,61.3
4,1891,1891-3,1,75,22,1,2,0,0,1,...,99.6,50.2,62.8,59.04,80.0,80.0,95.0,97.4,48.9,61.3


In [170]:
df_train = df_train_raw.groupby(['key']).mean().reset_index()

In [175]:
df2 = df_train_raw.groupby('key')[['xx1','xx2','xx3','xx4','xx5']].apply(lambda x : x[-2:-1]).reset_index()
df2.columns = ['key','level','xx1_2','xx2_2','xx3_2','xx4_2','xx5_2']
df2.drop(['level'], axis = 1, inplace = True)
df_train = df_train.merge(df2, on = 'key', how = 'left')

df3 = df_train_raw.groupby('key')[['xx1','xx2','xx3','xx4','xx5']].apply(lambda x : x[-3:-2]).reset_index()
df3.columns = ['key','level','xx1_3','xx2_3','xx3_3','xx4_3','xx5_3']
df3.drop(['level'], axis = 1, inplace = True)
df_train = df_train.merge(df3, on = 'key', how = 'left')

df4 = df_train_raw.groupby('key')[['xx1','xx2','xx3','xx4','xx5']].apply(lambda x : x[-4:-3]).reset_index()
df4.columns = ['key','level','xx1_4','xx2_4','xx3_4','xx4_4','xx5_4']
df4.drop(['level'], axis = 1, inplace = True)
df_train = df_train.merge(df4, on = 'key', how = 'left')

In [176]:
df1 = df_train_raw.groupby('key').tail(5).reset_index()
ts = df1.groupby(["key"]).agg({'xx1':['median','min','max'],'xx2':['median','min','max'],
                         'xx3':['median','min','max'], 'xx4':['median','min','max'],
                         'xx5':['median','min','max']}).reset_index()
ts.columns = ['key','xx1_10_median','xx1_10_min','xx1_10_max',
             'xx2_10_median','xx2_10_min','xx2_10_max',
             'xx3_10_median','xx3_10_min', 'xx3_10_max',
             'xx4_10_median','xx4_10_min','xx4_10_max',
             'xx5_10_median','xx5_10_min','xx5_10_max']


In [177]:
df_train = df_train.merge(ts, on = 'key', how = 'left')

In [178]:
stats = df_train_raw.groupby(['patient_id','key']).agg({'xx1':['last'],'xx2':['last'],
                                                        'xx3':['last'],'xx4':['last'],
                                                        'xx5':['last']}).reset_index()
stats.columns = ['patient_id','key','xx1_last','xx2_last',
                 'xx3_last','xx4_last','xx5_last']
stats.drop(['patient_id'], axis = 1, inplace = True)
df_train = df_train.merge(stats, on = 'key', how = 'left')

In [179]:
df_train.head()

,key,patient_id,gender,age,x1,x2,x3,x4,x5,x6,...,xx4_10_min,xx4_10_max,xx5_10_median,xx5_10_min,xx5_10_max,xx1_last,xx2_last,xx3_last,xx4_last,xx5_last
0,0-28,0,0,72,39,5,1,0,0,0,...,67.0,75.0,86.0,84.0,91.0,79.0,100.0,112.0,71.0,88.0
1,1-10,1,1,64,55,9,5,1,0,1,...,58.0,58.0,73.0,72.0,73.0,71.0,100.0,109.0,58.0,73.0
2,1-11,1,1,64,55,9,5,1,0,1,...,60.0,62.0,77.0,77.0,78.0,73.0,100.0,117.0,61.0,77.0
3,1-14,1,1,64,55,9,5,1,0,1,...,69.0,71.0,82.0,80.0,84.0,119.0,100.0,116.0,70.0,82.0
4,1-15,1,1,64,55,9,5,1,0,1,...,85.0,91.0,106.0,99.0,107.0,190.0,100.0,133.0,85.0,99.0


In [180]:
df_train.columns

Index(['key', 'patient_id', 'gender', 'age', 'x1', 'x2', 'x3', 'x4', 'x5',
       'x6', 'xx1', 'xx2', 'xx3', 'xx4', 'xx5', 'y_mean_MAP', 'y_mean_HR',
       'xx1_roll', 'xx2_roll', 'xx3_roll', 'xx4_roll', 'xx5_roll', 'xx1_2',
       'xx2_2', 'xx3_2', 'xx4_2', 'xx5_2', 'xx1_3', 'xx2_3', 'xx3_3', 'xx4_3',
       'xx5_3', 'xx1_4', 'xx2_4', 'xx3_4', 'xx4_4', 'xx5_4', 'xx1_10_median',
       'xx1_10_min', 'xx1_10_max', 'xx2_10_median', 'xx2_10_min', 'xx2_10_max',
       'xx3_10_median', 'xx3_10_min', 'xx3_10_max', 'xx4_10_median',
       'xx4_10_min', 'xx4_10_max', 'xx5_10_median', 'xx5_10_min', 'xx5_10_max',
       'xx1_last', 'xx2_last', 'xx3_last', 'xx4_last', 'xx5_last'],
      dtype='object')

In [181]:
#drop target column
df_train.drop(['y_mean_HR','y_mean_MAP'], axis = 1, inplace = True)

In [182]:
df_train.shape

(44949, 55)

In [183]:
valid_idx = range(int(df_train.shape[0]*80/100), df_train.shape[0])
train_idx = range(int(df_train.shape[0]*80/100))

In [184]:
#specify Target
y_mean_MAP = df_train_raw[['y_mean_MAP','key']].groupby('key').mean().reset_index().drop(['key'], axis = 1)
y_mean_HR = df_train_raw[['y_mean_HR','key']].groupby('key').mean().reset_index().drop(['key'], axis = 1)

In [185]:
#Train Test Split
X_map_train = df_train.loc[train_idx,:]
X_map_val = df_train.loc[valid_idx,:]
y_map_train = y_mean_MAP.loc[train_idx]
y_map_val = y_mean_MAP.loc[valid_idx]

In [186]:
X_hr_train = df_train.loc[train_idx,:]
X_hr_val = df_train.loc[valid_idx,:]
y_hr_train = y_mean_HR.loc[train_idx]
y_hr_val = y_mean_HR.loc[valid_idx]

In [187]:
#Sanity Check
print(X_map_train.key.isin(X_map_val.key).all())
print(X_hr_train.key.isin(X_hr_val.key).all())

False
False


In [188]:
X_map_train = X_map_train.drop(['key','patient_id'],axis = 1)
X_map_val = X_map_val.drop(['key','patient_id'],axis = 1)
X_hr_train = X_hr_train.drop(['key','patient_id'],axis = 1)
X_hr_val = X_hr_val.drop(['key','patient_id'],axis = 1)

In [189]:
X_map_train.shape

(35959, 53)

In [190]:
X_map_train.head()

,gender,age,x1,x2,x3,x4,x5,x6,xx1,xx2,...,xx4_10_min,xx4_10_max,xx5_10_median,xx5_10_min,xx5_10_max,xx1_last,xx2_last,xx3_last,xx4_last,xx5_last
0,0,72,39,5,1,0,0,0,80.166667,99.733333,...,67.0,75.0,86.0,84.0,91.0,79.0,100.0,112.0,71.0,88.0
1,1,64,55,9,5,1,0,1,72.100000,99.800000,...,58.0,58.0,73.0,72.0,73.0,71.0,100.0,109.0,58.0,73.0
2,1,64,55,9,5,1,0,1,71.500000,99.966667,...,60.0,62.0,77.0,77.0,78.0,73.0,100.0,117.0,61.0,77.0
3,1,64,55,9,5,1,0,1,99.133333,100.000000,...,69.0,71.0,82.0,80.0,84.0,119.0,100.0,116.0,70.0,82.0
4,1,64,55,9,5,1,0,1,150.200000,100.000000,...,85.0,91.0,106.0,99.0,107.0,190.0,100.0,133.0,85.0,99.0


### Deep learning model y_mean_MAP

In [191]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.layers import Dropout
from keras import optimizers

In [194]:
model = Sequential()
model.add(Dense(53, input_dim=53, kernel_initializer='normal', activation='relu'))
model.add(Dense(45, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
# Compile model

adam = optimizers.Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='mse', optimizer=adam, metrics=['mse','mae'])
history = model.fit(X_map_train, y_map_train, epochs=100, batch_size=50,  verbose=1, validation_data = (X_map_val,y_map_val))

Train on 35959 samples, validate on 8990 samples
Epoch 1/100
35959/35959 [==============================] - 1s 20us/step - loss: 190.4826 - mse: 190.4826 - mae: 6.6881 - val_loss: 25.9060 - val_mse: 25.9060 - val_mae: 3.2371
Epoch 2/100
35959/35959 [==============================] - 1s 15us/step - loss: 25.4516 - mse: 25.4516 - mae: 3.2230 - val_loss: 24.2841 - val_mse: 24.2841 - val_mae: 3.0246
Epoch 3/100
35959/35959 [==============================] - 1s 16us/step - loss: 24.4761 - mse: 24.4761 - mae: 3.1024 - val_loss: 23.5172 - val_mse: 23.5172 - val_mae: 2.9508
Epoch 4/100
35959/35959 [==============================] - 1s 16us/step - loss: 23.5839 - mse: 23.5839 - mae: 2.9913 - val_loss: 22.5974 - val_mse: 22.5974 - val_mae: 2.8225
Epoch 5/100
35959/35959 [==============================] - 1s 16us/step - loss: 22.7389 - mse: 22.7389 - mae: 2.8834 - val_loss: 22.5082 - val_mse: 22.5082 - val_mae: 2.8493s - loss: 22.5303 - mse: 22.5303 - mae: 2.89
Epoch 6/100
35959/35959 [==========

35959/35959 [==============================] - 1s 15us/step - loss: 21.1233 - mse: 21.1233 - mae: 2.7098 - val_loss: 21.1224 - val_mse: 21.1224 - val_mae: 2.5710
Epoch 48/100
35959/35959 [==============================] - 1s 15us/step - loss: 21.1026 - mse: 21.1026 - mae: 2.7061 - val_loss: 20.8160 - val_mse: 20.8160 - val_mae: 2.5765
Epoch 49/100
35959/35959 [==============================] - 1s 15us/step - loss: 21.1431 - mse: 21.1431 - mae: 2.7207 - val_loss: 20.7871 - val_mse: 20.7871 - val_mae: 2.5163
Epoch 50/100
35959/35959 [==============================] - 1s 16us/step - loss: 21.0897 - mse: 21.0897 - mae: 2.7039 - val_loss: 20.7816 - val_mse: 20.7816 - val_mae: 2.5897
Epoch 51/100
35959/35959 [==============================] - 1s 16us/step - loss: 21.1026 - mse: 21.1026 - mae: 2.6973 - val_loss: 21.0956 - val_mse: 21.0956 - val_mae: 2.6096
Epoch 52/100
35959/35959 [==============================] - 1s 16us/step - loss: 21.2294 - mse: 21.2294 - mae: 2.7232 - val_loss: 20.8537 

35959/35959 [==============================] - 1s 16us/step - loss: 20.8779 - mse: 20.8779 - mae: 2.6797 - val_loss: 20.7570 - val_mse: 20.7570 - val_mae: 2.5222
Epoch 94/100
35959/35959 [==============================] - 1s 15us/step - loss: 20.8570 - mse: 20.8570 - mae: 2.6589 - val_loss: 21.3259 - val_mse: 21.3259 - val_mae: 2.7645
Epoch 95/100
35959/35959 [==============================] - 1s 15us/step - loss: 20.9261 - mse: 20.9261 - mae: 2.6703 - val_loss: 21.0046 - val_mse: 21.0046 - val_mae: 2.6810
Epoch 96/100
35959/35959 [==============================] - 1s 15us/step - loss: 20.9847 - mse: 20.9847 - mae: 2.6941 - val_loss: 20.7744 - val_mse: 20.7744 - val_mae: 2.5961
Epoch 97/100
35959/35959 [==============================] - 1s 15us/step - loss: 20.7579 - mse: 20.7579 - mae: 2.6465 - val_loss: 20.8822 - val_mse: 20.8822 - val_mae: 2.6297
Epoch 98/100
35959/35959 [==============================] - 1s 15us/step - loss: 20.8847 - mse: 20.8847 - mae: 2.6703 - val_loss: 20.7028 

In [195]:
y_map_val_pred = model.predict(X_map_val)
r2_score(y_map_val,y_map_val_pred)

0.889880042804945

### Deep learning Model y_mean_HR

In [160]:
model_hr = Sequential()
model_hr.add(Dense(38, input_dim=38, kernel_initializer='normal', activation='relu'))
model_hr.add(Dense(33, kernel_initializer='normal', activation='relu'))
model_hr.add(Dense(1, kernel_initializer='normal'))
    # Compile model
adam = optimizers.Adam(learning_rate=0.003, beta_1=0.9, beta_2=0.999, amsgrad=False)
model_hr.compile(loss='mse', optimizer=adam, metrics=['mse','mae'])
history_hr = model_hr.fit(X_hr_train, y_hr_train, epochs=100, batch_size=50,  verbose=1, validation_data = (X_hr_val,y_hr_val))

Train on 35959 samples, validate on 8990 samples
Epoch 1/100
35959/35959 [==============================] - 1s 19us/step - loss: 152.0473 - mse: 152.0473 - mae: 5.3234 - val_loss: 12.1353 - val_mse: 12.1353 - val_mae: 1.7582
Epoch 2/100
35959/35959 [==============================] - 1s 15us/step - loss: 15.4673 - mse: 15.4673 - mae: 1.9997 - val_loss: 21.1086 - val_mse: 21.1086 - val_mae: 3.4918
Epoch 3/100
35959/35959 [==============================] - 1s 16us/step - loss: 16.0179 - mse: 16.0179 - mae: 2.1351 - val_loss: 17.4095 - val_mse: 17.4095 - val_mae: 2.9049
Epoch 4/100
35959/35959 [==============================] - 1s 16us/step - loss: 15.9084 - mse: 15.9084 - mae: 2.1424 - val_loss: 13.6575 - val_mse: 13.6575 - val_mae: 2.1951
Epoch 5/100
35959/35959 [==============================] - 1s 16us/step - loss: 15.8705 - mse: 15.8705 - mae: 2.1356 - val_loss: 13.1756 - val_mse: 13.1756 - val_mae: 2.0790
Epoch 6/100
35959/35959 [==============================] - 1s 16us/step - loss:

35959/35959 [==============================] - 1s 15us/step - loss: 14.6570 - mse: 14.6570 - mae: 1.9216 - val_loss: 11.3461 - val_mse: 11.3461 - val_mae: 1.6099
Epoch 48/100
35959/35959 [==============================] - 1s 15us/step - loss: 14.7184 - mse: 14.7185 - mae: 1.9290 - val_loss: 11.4969 - val_mse: 11.4969 - val_mae: 1.6749
Epoch 49/100
35959/35959 [==============================] - 1s 15us/step - loss: 14.8029 - mse: 14.8029 - mae: 1.9296 - val_loss: 11.3131 - val_mse: 11.3131 - val_mae: 1.5956
Epoch 50/100
35959/35959 [==============================] - 1s 15us/step - loss: 14.8536 - mse: 14.8536 - mae: 1.9607 - val_loss: 11.5517 - val_mse: 11.5517 - val_mae: 1.6853
Epoch 51/100
35959/35959 [==============================] - 1s 15us/step - loss: 14.7768 - mse: 14.7768 - mae: 1.9418 - val_loss: 14.0779 - val_mse: 14.0779 - val_mae: 2.3553
Epoch 52/100
35959/35959 [==============================] - 1s 15us/step - loss: 14.9592 - mse: 14.9592 - mae: 1.9716 - val_loss: 13.3492 

Epoch 94/100
35959/35959 [==============================] - 1s 15us/step - loss: 14.5227 - mse: 14.5227 - mae: 1.8868 - val_loss: 11.9794 - val_mse: 11.9794 - val_mae: 1.8408
Epoch 95/100
35959/35959 [==============================] - 1s 15us/step - loss: 14.5652 - mse: 14.5652 - mae: 1.8970 - val_loss: 11.3784 - val_mse: 11.3784 - val_mae: 1.6604
Epoch 96/100
35959/35959 [==============================] - 1s 16us/step - loss: 14.4482 - mse: 14.4482 - mae: 1.8835 - val_loss: 11.9070 - val_mse: 11.9070 - val_mae: 1.8214
Epoch 97/100
35959/35959 [==============================] - 1s 16us/step - loss: 14.4980 - mse: 14.4980 - mae: 1.8894 - val_loss: 12.7284 - val_mse: 12.7284 - val_mae: 2.0731
Epoch 98/100
35959/35959 [==============================] - 1s 15us/step - loss: 14.6966 - mse: 14.6966 - mae: 1.9406 - val_loss: 11.5969 - val_mse: 11.5969 - val_mae: 1.7252
Epoch 99/100
35959/35959 [==============================] - 1s 15us/step - loss: 14.5792 - mse: 14.5792 - mae: 1.9147 - val_l

In [161]:
y_hr_val_pred = model_hr.predict(X_hr_val)
r2_score(y_hr_val,y_hr_val_pred)

0.9528174450328679

### XGBoost on y_mean_MAP

In [ ]:
#feature selection
X_map_train.columns

In [ ]:
# X_map_train = X_map_train[['gender','age','x1','x2','xx3','xx4','xx5','xx3_mean','xx3_last',
#                    'xx4_mean','xx4_last','xx5_mean','xx5_last']]
# X_map_val = X_map_val[['gender','age','x1','x2','xx3','xx4','xx5','xx3_mean','xx3_last',
#                    'xx4_mean','xx4_last','xx5_mean','xx5_last']]
X_map_train = X_map_train.drop(['key','patient_id'],axis = 1)
X_map_val = X_map_val.drop(['key','patient_id'],axis = 1)

In [ ]:
params = {'learning_rate':[0.03,0.01,0.1,0.5],
            'min_child_weight':[4,5], 
          'gamma':[i/10.0 for i in range(3,6)],  
          'subsample':[i/10.0 for i in range(6,11)],
            'n_estimator':[200,500,1000,1500], 
            'max_depth': [2,3,4]}

xgb_map = xgb.XGBRegressor(nthread=-1) 

grid_map = GridSearchCV(xgb_map, params)
grid_map.fit(X_map_train,y_map_train)

In [ ]:
print(r2_score(y_map_val, grid_map.best_estimator_.predict(X_map_val)))

In [ ]:
xgb_map = xgb.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.01,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=3000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)
xgb_map.fit(X_map_train,y_map_train)

In [ ]:
y_map_val_pred = xgb_map.predict(X_map_val)
r2_score(y_map_val,y_map_val_pred)

### XGBoost on y_mean_HR

In [ ]:
#feature selection
X_hr_train.columns

In [ ]:
X_hr_train = X_hr_train[['age','xx4','xx5','xx4_mean','xx4_last',
                          'xx5_mean','xx5_last','xx1','xx1_mean','xx1_last']]
X_hr_val = X_hr_val[['age','xx4','xx5','xx4_mean','xx4_last',
                          'xx5_mean','xx5_last','xx1','xx1_mean','xx1_last']]

In [ ]:
params = {'learning_rate':[0.03,0.01,0.1,0.5],  
          'subsample':[i/10.0 for i in range(6,11)],
            'n_estimator':[200,500,1000,1500], 
            'max_depth': [2,3,4]}

xgb_hr = xgb.XGBRegressor(nthread=-1) 

grid_hr = GridSearchCV(xgb_hr, params)
grid_hr.fit(X_hr_train,y_hr_train)

In [ ]:
print(r2_score(y_hr_val, grid_hr.best_estimator_.predict(X_hr_val)))

In [ ]:
xgb_hr = xgb.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.01,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=1000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)
xgb_hr.fit(X_hr_train,y_hr_train)

In [ ]:
y_hr_val_pred = xgb_hr.predict(X_hr_val)
r2_score(y_hr_val,y_hr_val_pred)

### Test

In [50]:
df_test_raw['xx1_roll'] = df_test_raw['xx1'].rolling(3).mean()
df_test_raw['xx2_roll'] = df_test_raw['xx2'].rolling(3).mean() 
df_test_raw['xx3_roll'] = df_test_raw['xx3'].rolling(3).mean() 
df_test_raw['xx4_roll'] = df_test_raw['xx4'].rolling(3).mean() 
df_test_raw['xx5_roll'] = df_test_raw['xx5'].rolling(3).mean() 

In [51]:
df_test = df_test_raw.groupby(['key']).mean().reset_index()

In [52]:
df_test.head()

,key,patient_id,gender,age,x1,x2,x3,x4,x5,x6,xx1,xx2,xx3,xx4,xx5,xx1_roll,xx2_roll,xx3_roll,xx4_roll,xx5_roll
0,1005-1,1005,0,80,41,4,2,1,0,0,57.866667,98.433333,126.566667,42.800000,71.466667,57.940476,98.416667,127.250000,43.190476,71.988095
1,1005-10,1005,0,80,41,4,2,0,0,0,75.766667,97.333333,126.933333,48.400000,71.866667,75.644444,97.077778,126.166667,48.377778,71.600000
2,1005-11,1005,0,80,41,4,2,1,0,0,65.666667,92.166667,109.000000,48.633333,65.600000,65.866667,92.488889,109.722222,48.655556,65.833333
3,1005-12,1005,0,80,41,4,2,1,0,0,64.600000,94.933333,124.200000,55.400000,75.133333,64.622222,94.688889,124.188889,55.444444,75.055556
4,1005-13,1005,0,80,41,4,2,0,0,0,73.800000,97.400000,134.733333,52.366667,77.200000,73.422222,97.322222,133.777778,52.100000,76.811111


In [53]:
df1 = df_test_raw.groupby('key').tail(5).reset_index()
ts = df1.groupby(["key"]).agg({'xx1':['mean','min'],'xx2':['mean','min'],
                         'xx3':['mean','min'], 'xx4':['mean','min'],
                         'xx5':['mean','min']}).reset_index()
ts.columns = ['key','xx1_10_mean','xx1_10_min',
             'xx2_10_mean','xx2_10_min',
             'xx3_10_mean','xx3_10_min',
             'xx4_10_mean','xx4_10_min',
             'xx5_10_mean','xx5_10_min']

In [54]:
df_test = df_test.merge(ts, on = 'key', how = 'left')

In [55]:
stats = df_test_raw.groupby(['patient_id','key']).agg({'xx1':['last'],'xx2':['last'],
                                                        'xx3':['last'],'xx4':['last'],
                                                        'xx5':['last']}).reset_index()
stats.columns = ['patient_id','key','xx1_last','xx2_last',
                 'xx3_last','xx4_last','xx5_last']
stats.drop(['patient_id'], axis = 1, inplace = True)
df_test = df_test.merge(stats, on = 'key', how = 'left')

In [56]:
df_test.head()

,key,patient_id,gender,age,x1,x2,x3,x4,x5,x6,...,xx3_10_min,xx4_10_mean,xx4_10_min,xx5_10_mean,xx5_10_min,xx1_last,xx2_last,xx3_last,xx4_last,xx5_last
0,1005-1,1005,0,80,41,4,2,1,0,0,...,110.0,37.8,35.0,63.4,60.0,55.0,99.0,116.0,36.0,61.0
1,1005-10,1005,0,80,41,4,2,0,0,0,...,120.0,48.8,46.0,71.8,68.0,71.0,100.0,124.0,46.0,68.0
2,1005-11,1005,0,80,41,4,2,1,0,0,...,102.0,47.4,46.0,64.4,62.0,65.0,89.0,105.0,46.0,62.0
3,1005-12,1005,0,80,41,4,2,1,0,0,...,97.0,48.0,40.0,68.4,58.0,68.0,97.0,108.0,47.0,68.0
4,1005-13,1005,0,80,41,4,2,0,0,0,...,132.0,53.4,52.0,77.6,76.0,76.0,99.0,132.0,52.0,76.0


In [57]:
df_test.head()

,key,patient_id,gender,age,x1,x2,x3,x4,x5,x6,...,xx3_10_min,xx4_10_mean,xx4_10_min,xx5_10_mean,xx5_10_min,xx1_last,xx2_last,xx3_last,xx4_last,xx5_last
0,1005-1,1005,0,80,41,4,2,1,0,0,...,110.0,37.8,35.0,63.4,60.0,55.0,99.0,116.0,36.0,61.0
1,1005-10,1005,0,80,41,4,2,0,0,0,...,120.0,48.8,46.0,71.8,68.0,71.0,100.0,124.0,46.0,68.0
2,1005-11,1005,0,80,41,4,2,1,0,0,...,102.0,47.4,46.0,64.4,62.0,65.0,89.0,105.0,46.0,62.0
3,1005-12,1005,0,80,41,4,2,1,0,0,...,97.0,48.0,40.0,68.4,58.0,68.0,97.0,108.0,47.0,68.0
4,1005-13,1005,0,80,41,4,2,0,0,0,...,132.0,53.4,52.0,77.6,76.0,76.0,99.0,132.0,52.0,76.0


In [58]:
test_map_pred = model.predict(df_test.drop(['key','patient_id'],axis = 1))

In [59]:
test_hr_pred = model_hr.predict(df_test.drop(['key','patient_id'],axis = 1))
#test_map_pred = model.predict(df_test)

In [60]:
df_test['y_mean_MAP'] = test_map_pred
df_test['y_mean_HR'] = test_hr_pred

In [61]:
df_submit = df_test[['key','y_mean_MAP','y_mean_HR']]

In [64]:
df_submit.head()

,key,y_mean_MAP,y_mean_HR
0,1005-1,64.612427,55.513851
1,1005-10,70.592316,75.771942
2,1005-11,64.253403,65.760674
3,1005-12,68.385178,62.693153
4,1005-13,77.248283,75.314697


In [71]:
df_submit = df_submit.groupby('key').mean()

In [72]:
df_submit.head()

,y_mean_MAP,y_mean_HR
key,,
1005-1,64.612427,55.513851
1005-10,70.592316,75.771942
1005-11,64.253403,65.760674
1005-12,68.385178,62.693153
1005-13,77.248283,75.314697


In [73]:
df_submit.to_csv("no_one_9.csv")